In [68]:
import sframe                                            # see below for install instruction
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
from scipy.stats import multivariate_normal
from copy import deepcopy
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [86]:
from em_utilities import *
from utilities import *

In [70]:
with open('arquivo_subset.json') as json_file:
    arquivos_json = json.load(json_file)
for arquivo in arquivos_json:
    arquivo['api_Arquivo_bn'] = arquivo['api_Arquivo_bn'].encode('utf-8').replace('\n',' ')
df = pd.DataFrame(arquivos_json)
df

,api_Arquivo_bn,api_Nome_do_arquivo_tg,ctr_id
0,89700-000 CONCORDIA /SC II II II li II II II I...,14010345000000042014022013.PDF,e7b1caef1a4214b070b10194d35725caaa093e6d
1,NA ITSEEO/GGRAN/DR/GT DATA DE POTAGEM 27/01/20...,14010345000000052014022013.PDF,2c8f7e2b456f7f5716132b00548430fee887db7a
2,"CORREIO< DESTINATÁRIO: kg, a DANI E RINALDI TR...",14010345000000072014022013.PDF,31a127b1b09a5ffae8c9ea921040049922d8348e
3,NA IT SEED/GGRANtDR/GT CORREIOS DESTINATÁRIO: ...,14010345000000112014022013.PDF,2de0586a4d8d03afa8a1f975a16f73955016731e
4,89560-000 VIDEIRA / SC IIII AR 340405364 AG II...,14010345000000142014022013.PDF,d0f55a2299497200428c9e8d9594d56afb0509c7
5,igi a CONTROLE NAP CORREIO< SEED/GGRAN/DR/GT D...,14010345000000172014022013.PDF,4b356b47c2a2cdc8df3438c06ed4f4e4f99e4979
6,SEED/GGRAN/DR/G0 DATA DE POSTAGEM 10102/2014 A...,14010345000000182014022013.PDF,c12a4e69ad3802b0a02a0afaafe81094ffcd7fb8
7,CORREIO< mar /gr a ia« 11 NOTIFICAÇÃO DE AUTUA...,14010345000000202014022013.PDF,7dfca19151197265b1de79d8f111a864e394a201
8,SEED/GGRAN/DR/GO DATA DE POSTAGEM CORREIO< 10(...,14010345000000192014022013.PDF,c41b61f5d2058daf55e7d11a814acc967510f360
9,"DATA DE ENTREGA 7.Y y N° DOCUM ENT,0 10E IDEDI...",14010345000000272014022013.PDF,de97a9f8ad2cd983dc43a2fdc526b2ce8e933800


In [71]:
df

,api_Arquivo_bn,api_Nome_do_arquivo_tg,ctr_id
0,89700-000 CONCORDIA /SC II II II li II II II I...,14010345000000042014022013.PDF,e7b1caef1a4214b070b10194d35725caaa093e6d
1,NA ITSEEO/GGRAN/DR/GT DATA DE POTAGEM 27/01/20...,14010345000000052014022013.PDF,2c8f7e2b456f7f5716132b00548430fee887db7a
2,"CORREIO< DESTINATÁRIO: kg, a DANI E RINALDI TR...",14010345000000072014022013.PDF,31a127b1b09a5ffae8c9ea921040049922d8348e
3,NA IT SEED/GGRANtDR/GT CORREIOS DESTINATÁRIO: ...,14010345000000112014022013.PDF,2de0586a4d8d03afa8a1f975a16f73955016731e
4,89560-000 VIDEIRA / SC IIII AR 340405364 AG II...,14010345000000142014022013.PDF,d0f55a2299497200428c9e8d9594d56afb0509c7
5,igi a CONTROLE NAP CORREIO< SEED/GGRAN/DR/GT D...,14010345000000172014022013.PDF,4b356b47c2a2cdc8df3438c06ed4f4e4f99e4979
6,SEED/GGRAN/DR/G0 DATA DE POSTAGEM 10102/2014 A...,14010345000000182014022013.PDF,c12a4e69ad3802b0a02a0afaafe81094ffcd7fb8
7,CORREIO< mar /gr a ia« 11 NOTIFICAÇÃO DE AUTUA...,14010345000000202014022013.PDF,7dfca19151197265b1de79d8f111a864e394a201
8,SEED/GGRAN/DR/GO DATA DE POSTAGEM CORREIO< 10(...,14010345000000192014022013.PDF,c41b61f5d2058daf55e7d11a814acc967510f360
9,"DATA DE ENTREGA 7.Y y N° DOCUM ENT,0 10E IDEDI...",14010345000000272014022013.PDF,de97a9f8ad2cd983dc43a2fdc526b2ce8e933800


In [72]:
tfidf_vectorizer = TfidfVectorizer()
x = tfidf_vectorizer.fit_transform(df['api_Arquivo_bn'])
print x

  (0, 16655)	0.1207839797
  (0, 12890)	0.135146061745
  (0, 6738)	0.0291939588432
  (0, 8863)	0.14950814379
  (0, 9204)	0.0476413838059
  (0, 6831)	0.14950814379
  (0, 17390)	0.135146061745
  (0, 11628)	0.0869246243837
  (0, 1413)	0.14950814379
  (0, 6814)	0.14950814379
  (0, 13449)	0.14950814379
  (0, 6213)	0.0246015826344
  (0, 11990)	0.0325705342128
  (0, 15399)	0.0258624124116
  (0, 12764)	0.0276918444621
  (0, 15457)	0.14950814379
  (0, 1545)	0.123550764115
  (0, 14837)	0.14950814379
  (0, 5545)	0.0332413875607
  (0, 8913)	0.0585829961656
  (0, 9493)	0.14950814379
  (0, 16607)	0.14950814379
  (0, 6185)	0.026536515603
  (0, 7482)	0.0507470432187
  (0, 6345)	0.0381318132332
  :	:
  (999, 12996)	0.0263644684846
  (999, 8721)	0.026259181803
  (999, 13915)	0.0258165992192
  (999, 8200)	0.0338068347361
  (999, 10438)	0.0279432021128
  (999, 14893)	0.0337004645474
  (999, 7411)	0.050611517505
  (999, 7054)	0.0254833670149
  (999, 15022)	0.02993090237
  (999, 13835)	0.0306719968711
  (999

In [73]:
x.shape

(1000, 17528)

In [80]:
feature_names = tfidf_vectorizer.get_feature_names()
for idx, feature in enumerate(feature_names):
    feature_names[idx] = feature.encode('utf-8')
print x[0, 16655]

0.1207839797


In [76]:
len(tfidf_vectorizer.get_feature_names())

17528

In [85]:
def arquivo_contem(conteudo, feature):
    return 1 if feature in conteudo else 0

In [82]:
cols_to_retain = feature_names
for feature in feature_names:
    df[feature] = df.apply(lambda row: arquivo_contem(row['api_Arquivo_bn'], feature), axis = 1)

/home/pedro.castro/anaconda2/envs/dato-env/lib/python2.7/site-packages/pandas/core/index.py:1122: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine
/home/pedro.castro/anaconda2/envs/dato-env/lib/python2.7/site-packages/pandas/core/index.py:1759: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self._engine.get_loc(key)
/home/pedro.castro/anaconda2/envs/dato-env/lib/python2.7/site-packages/pandas/core/index.py:1790: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self._engine.get_value(s, k)


KeyboardInterrupt: 

In [ ]:
df